In [1]:
import pexpect
import json
import os

class LeanServer:
    def __init__(self):
        # Get the path where you download repl
        path_to_repl = os.environ.get('PATH_TO_LEAN_REPL')

        # Run the command
        self.proc = pexpect.spawn(
            "lake env lean --run REPL/Main.lean", 
            cwd=path_to_repl, 
            encoding="utf-8")

    def run_code(self, code, env=None, verbose=False):
        if env:
            command = (
                json.dumps(dict(cmd=code, env=env))
            )  # [1:-1] removes single quotes
        else:
            command = (
                '{ "cmd" : "' + repr(code)[1:-1] + '" }'
            )  # [1:-1] removes single quotes

        if verbose: print(command)
        self.proc.sendline(command)
        self.proc.expect_exact(command + "\r\n")
        self.proc.sendline()
        self.proc.expect_exact("\r\n")
        try:
            index = self.proc.expect('env": \d+\}', timeout=20)
            output = self.proc.before + self.proc.match.group()
            if verbose: print(output)
            return json.loads(output)
            
        except pexpect.exceptions.TIMEOUT:
            raise pexpect.exceptions.TIMEOUT

In [5]:
def get_goal(state):
    goal = None
    for msg in state['messages']:
        if msg['data'].startswith('unsolved goals\n'):
            goal = '\n'.join(msg['data'].split('\n')[1:])
            
        elif msg['severity'] == 'error':
            return None
            
    return goal

In [19]:
from pprint import pprint

code = """
import Mathlib.Data.Nat.Prime

theorem test_thm (m n : Nat) (h : m.Coprime n) : m.gcd n = 1 := by {}
"""

lean = LeanServer()
state = lean.run_code(code)
lean.proc.close()
pprint(state)

{'env': 0,
 'messages': [{'data': 'unsolved goals\n'
                       'm n : ℕ\n'
                       'h : Nat.Coprime m n\n'
                       '⊢ Nat.gcd m n = 1',
               'endPos': {'column': 69, 'line': 4},
               'pos': {'column': 68, 'line': 4},
               'severity': 'error'}]}


In [20]:
# code = """
# import Mathlib.Data.Nat.Prime

# theorem test_thm (m n : Nat) (h : m.Coprime n) : m.gcd n = 1 := by 

# rw [← h.gcd_eq_one]
# """

# lean = LeanServer()
# state = lean.run_code(code)
# lean.proc.close()

# pprint(state)

In [21]:
not state.get('sorries') and not state.get('messages')

False

In [17]:
# Load model and tokenizer
import os
import transformers

model_name = 'wellecks/llmstep-mathlib4-pythia2.8b'
model = transformers.GPTNeoXForCausalLM.from_pretrained(model_name)

tokenizer = transformers.GPTNeoXTokenizerFast.from_pretrained(model_name)
os.environ['TOKENIZERS_PARALLELISM'] = 'true'  # prevents an annoying warning

def generate(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    out = model.generate(
        input_ids,
        max_new_tokens=256,
        pad_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(out[0][input_ids.shape[1]:], skip_special_tokens=True)
    return text

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
# Generate a next step
prompt = f"[GOAL]{get_goal(state)}[PROOFSTEP]"
print(prompt)

[GOAL]m n : ℕ
h : Nat.Coprime m n
⊢ Nat.gcd m n = 1[PROOFSTEP]


In [28]:
next_step = generate(prompt)
print(next_step)

rw [← gcd_eq_gcd_ab, gcd_comm]


In [29]:
code = """
import Mathlib.Data.Nat.Prime

theorem test_thm (m n : Nat) (h : m.Coprime n) : m.gcd n = 1 := by 

""" + next_step

lean = LeanServer()
state = lean.run_code(code)
lean.proc.close()

pprint(state)

{'env': 0,
 'messages': [{'data': "unknown identifier 'gcd_eq_gcd_ab'",
               'endPos': {'column': 19, 'line': 6},
               'pos': {'column': 6, 'line': 6},
               'severity': 'error'},
              {'data': "tactic 'rewrite' failed, equality or iff proof "
                       'expected\n'
                       '  ?m.45\n'
                       'm n : ℕ\n'
                       'h : Nat.Coprime m n\n'
                       '⊢ Nat.gcd m n = 1',
               'endPos': {'column': 19, 'line': 6},
               'pos': {'column': 4, 'line': 6},
               'severity': 'error'}]}


In [39]:
"""
Reference: https://github.com/wellecks/ntptutorial/blob/main/partI_nextstep/ntp_python/proofsearch_pylean.py
"""

# Utilities for interacting with Lean and proof search

from pylean import LeanServer
import torch
import heapq
import concurrent
import transformers
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from typing import List, Tuple
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  


def is_done(state):
    return not state.get('sorries') and not state.get('messages')


def get_goal(state):
    goal = None
    msgs = state.get('messages')

    if msgs:
        for msg in msgs:
            
            if msg['data'].startswith('unsolved goals\n'):
                goal = '\n'.join(msg['data'].split('\n')[1:])
                
            elif msg['severity'] == 'error':
                return None
    return goal


def get_errors(state):
    return state['messages']


def parse_step(step):
    step = step.replace('<|endoftext|>', '')
    return step


def format_code(header, statement, steps_so_far, next_step):
    return header + (statement.replace(" {}", "") + '\n' + '\n'.join(steps_so_far + [next_step]))
    

def run_code(code):
    lean = LeanServer()
    out = lean.run_code(code)
    lean.proc.close()
    del lean
    return out


def sequence_scores(out, prompt_length, model, tokenizer):
    # Returns each output sequence's log probability normalized by the number of tokens.
    # An output sequence is defined as the tokens after the prompt up to and including eos.
    text = tokenizer.batch_decode(out.sequences)
    input_ids = tokenizer(
        text, return_tensors="pt", padding='longest', truncation=True
    ).to(model.device)
    with torch.no_grad():
        out = model(**input_ids)
        probs = torch.log_softmax(out.logits, dim=-1).detach()
        probs = probs[:, :-1, :]
        input_ids_shifted = input_ids.input_ids[:, 1:]
        log_probs = torch.gather(probs, 2, input_ids_shifted[:, :, None]).squeeze(-1)
        log_probs = log_probs[:, prompt_length:]
        up_to_eos_mask = (input_ids_shifted[:,prompt_length:].eq(
            tokenizer.eos_token_id).cumsum(1).cumsum(1) <= 1).type(log_probs.dtype)
        normalized_sequence_scores = (log_probs * up_to_eos_mask).sum(1) / up_to_eos_mask.sum(1)
    return normalized_sequence_scores


def generate(prompt, model, tokenizer, temperatures, num_samples) -> Tuple[List[str], List[float]]:
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(model.device)
    texts = []
    scores = []
    with torch.no_grad():
        # Does beam search at temp 0.0, otherwise temperature sampling.
        for temp in temperatures:
            decoding_params = dict(
                max_new_tokens=256,
                do_sample=temp > 0,
                temperature=temp,
                pad_token_id=tokenizer.eos_token_id,
                num_return_sequences=num_samples,
                return_dict_in_generate=True,
                output_scores=True,
            )
            if temp == 0.0:
                decoding_params['num_beams'] = num_samples
            out = model.generate(
                input_ids, **decoding_params
            )
            
            texts.extend(tokenizer.batch_decode(
                out.sequences[:,input_ids.shape[1]:],
                skip_special_tokens=True
            ))
            scores_ = sequence_scores(
                out=out, 
                prompt_length=input_ids.shape[1], 
                model=model, 
                tokenizer=tokenizer
            )
            scores.extend(scores_.view(-1).tolist())

    texts, scores = _unique_sorted(texts, scores)
    return texts, scores


def _unique_sorted(texts, scores):
    texts_, scores_ = [], []
    for t, s in sorted(zip(texts, scores), key=lambda x: -x[1]):
        if t not in texts_:
            texts_.append(t)
            scores_.append(s)
    return texts_, scores_


def _print_type_checked_candidates(results):
    print('--- type-checked candidates:\n\t' + '\n\t'.join(
        '(%.3f) %s' % (step_score, step) 
        for state, step, step_score in results if (
        get_goal(state) is not None or is_done(state))
    ))


def _print_current(theorem_statement, steps):
    print('--- current:\n\t%s\n\t%s' % (
        theorem_statement.replace('{}', ''),
        '\n\t'.join(steps)) 
    )


def best_first_search(model, tokenizer, header, statement, max_iters, 
                      temperatures, num_samples, verbose=False) -> dict:
    
    goal = get_goal(run_code(header + statement))
    if goal is None:
        return {
            'theorem_statement': statement, 
            'success': False, 
            'msg': run_code(header + statement)
        }

    # Score, steps-so-far, goal state
    queue = [(0.0, [], goal)]
    visited = set()
    while len(queue) > 0 and max_iters > 0:
        # Dequeue the tuple with minimum score
        score, steps, goal = heapq.heappop(queue)
        visited.add(goal)
        if verbose:
            _print_current(statement, steps)

        # Generate next-step candidates
        prompt = f"[GOAL]{goal}[PROOFSTEP]"
        step_cands, step_scores = generate(
            prompt, 
            model, 
            tokenizer, 
            temperatures=temperatures, 
            num_samples=num_samples
        )

        # Run type checking in parallel via futures. 
        with ThreadPoolExecutor(max_workers=16) as executor:
            # We need to save the step and score associated to each future.
            future2step = {}
            for step, step_score in zip(step_cands, step_scores):
                code = format_code(header, statement, steps, step)
                future = executor.submit(run_code, **dict(code=code))
                future2step[future] = (step, step_score)

            # Collect the type checking results as they complete.
            results = []
            for future in tqdm(concurrent.futures.as_completed(future2step.keys()), total=len(future2step)):
                result = future.result()
                results.append((result, *future2step[future]))

        if verbose:
            _print_type_checked_candidates(results)
        for state, step, step_score in results:
            # Stop if we have found a complete proof.
            if is_done(state):
                return {
                    'theorem_statement': statement, 
                    'proof': steps + [step], 
                    'state': state,
                    'score': score - step_score,
                    'success': True
                }
            goal_cand = get_goal(state)
            # Add new candidates to the queue.
            if goal_cand is not None and goal_cand not in visited:
                # Score is normalized negative log probability summed across steps
                new_score = (score - step_score)
                heapq.heappush(
                    queue, (new_score, steps+[step], goal_cand)
                )
        
        max_iters -= 1

    return {'theorem_statement': statement, 'success': False}


def _save(results):
    from datetime import datetime
    import json
    now = datetime.now()
    dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
    output_file = 'results__%s.json' % (dt_string)
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=4)
        print(output_file)


def load_model(model_name):
    model = transformers.GPTNeoXForCausalLM.from_pretrained(model_name)
    tokenizer = transformers.GPTNeoXTokenizerFast.from_pretrained(model_name)
    model.eval()
    return model, tokenizer


# if __name__ == '__main__':
#     model, tokenizer = load_model('wellecks/llmstep-mathlib4-pythia2.8b')

#     evaluation_theorems = [
#         """theorem thm1 (a b c : Nat) : a + b = c → a ≤ c := by {}""",
#         """theorem thm2 (x y : ℝ) : x < y → 0 < y - x := by {}""",
#         """theorem thm3 (n : Nat) : n ≥ 0 := by {}""",
#         """theorem thm4 (x y z : ℝ) : x ≤ y → y ≤ z → x ≤ z := by {}""",
#         """theorem thm5 (m n : Nat) (h : m.coprime n) : m.gcd n = 1 := by {}""",
#         """theorem thm6: r ⊆ s → s ⊆ t → r ⊆ t := by {}""",
#         """theorem thm7 (f : ℕ → ℕ) : Monotone f → ∀ n, f n ≤ f (n + 1) := by {}""",
#         """theorem thm8 (c : ℝ) : Injective fun x => x + c := by {}""",
#         """theorem thm9 (p q : Prop) : (p ∧ q) → ¬(¬p ∨ ¬q) := by {}""",
#         """theorem thm10 (A B : Set ℕ) : A ⊆ B → ∀ n, n ∈ A → n ∈ B := by {}""",
#         """theorem thm11 (injg : Injective g) (injf : Injective f) : Injective fun x => g (f x) := by {}""",
#         """theorem thm12 (a b : ℕ) (h : a ≤ b) : a * (a + 1) ≤ b * (b + 1) := by {}""",
#         """theorem thm13 (a b : ℕ) (h : a ≠ b) : a * 2 ≠ b * 2 := by {}""",
#     ]
        
#     # Shared header for the theorems above
#     header = """import Mathlib.Data.Nat.Factorization.Basic
#     import Mathlib.Data.Nat.Prime
#     import Mathlib.Data.Real.Basic
    
#     open BigOperators
#     open Function
#     variable {α : Type _} (r s t : Set α)
    
#     """

#     results = []
#     for theorem in evaluation_theorems:
#         result = best_first_search(
#             model, tokenizer, header, theorem, 
#             max_iters=32,
#             temperatures=[0.5],
#             num_samples=16
#         )
#         print(result)
#         print('\n-----\n')
#         results.append(result)

#     print(len([x for x in results if x['success']])/len(results))
#     _save(results)

In [33]:
transformers.set_seed(43)

def prove_simple(model, tokenizer, header, theorem_statement, search_budget):
    success = False

    code = header + theorem_statement
    steps = []
    proof = ''

    for i in range(search_budget):
        print("== Current (%d): " % i, theorem_statement[:-3] + '\n' + proof, sep='\n')

        # Run the code (header + proof-so-far)
        state = run_code(code)
        
        # Stop if the proof is complete.
        if is_done(state):
            success = True
            break

        # Get the new state.
        goal_candidate = get_goal(state)
        if goal_candidate is None:
            print("-- Error: backtracking")
            steps = steps[:-1]
        else:
            goal = goal_candidate

        print("-- Goal: ", goal, sep='\n')

        # Generate a next-step
        prompt = f"[GOAL]{goal}[PROOFSTEP]"
        texts, _= generate(prompt, model, tokenizer, temperatures=[0.5], num_samples=1)
        step = parse_step(texts[0])

        # Add the next-step to the proof-so-far
        steps.append(step)
        proof = '\n'.join(steps)
        code = header + theorem_statement.replace(" {}", "") + '\n' + proof
        print()

    if success:
        print("\nSUCCESS!")
    else:
        print("\nFAILED")
    
    print(theorem_statement.replace(" {}", ""))
    print ('  ' + proof.replace('\n', '\n  '))
    
    return {'theorem_statement': theorem_statement, 'proof': proof, 'success': success}

In [34]:
header = """
import Mathlib.Data.Nat.Prime

"""
theorem_statement = """theorem thm1 (a b c : Nat) : a + b = c → a ≤ c := by {}"""

out = prove_simple(
    model, 
    tokenizer,
    header, 
    theorem_statement, 
    search_budget=100
)

== Current (0): 
theorem thm1 (a b c : Nat) : a + b = c → a ≤ c := by

-- Goal: 
a b c : ℕ
⊢ a + b = c → a ≤ c

== Current (1): 
theorem thm1 (a b c : Nat) : a + b = c → a ≤ c := by
rintro rfl
-- Goal: 
a b : ℕ
⊢ a ≤ a + b

== Current (2): 
theorem thm1 (a b c : Nat) : a + b = c → a ≤ c := by
rintro rfl
exact le_add_left _ _
-- Error: backtracking
-- Goal: 
a b : ℕ
⊢ a ≤ a + b

== Current (3): 
theorem thm1 (a b c : Nat) : a + b = c → a ≤ c := by
rintro rfl
apply Nat.le_add_right

SUCCESS!
theorem thm1 (a b c : Nat) : a + b = c → a ≤ c := by
  rintro rfl
  apply Nat.le_add_right


In [35]:
prompt = '[GOAL]m n : ℕ\nh : Nat.coprime m n\n⊢ Nat.gcd m n = 1[PROOFSTEP]'

texts, scores = generate(prompt, model, tokenizer, temperatures=[0.0], num_samples=5)

for text, score in zip(texts, scores):
    print('%.3f' % score, text, sep='\t')

/localscratch/hsun409/anaconda3/envs/jepa/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


-0.277	rw [Nat.coprime, gcd_comm] at h
-0.279	rw [← h.gcd_eq_one]
-0.302	rw [← h.gcd_eq_one, Nat.gcd_comm]
-0.326	rw [← h.gcd_eq_one, gcd_comm]
-0.367	rw [← h.gcd_eq_right_iff_dvd]


In [40]:
best_first_search(
    model, tokenizer, header, theorem_statement, 
    max_iters=32,
    num_samples=4,
    temperatures=[0.0],
    verbose=True
)

--- current:
	theorem thm1 (a b c : Nat) : a + b = c → a ≤ c := by 
	


100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.18s/it]


--- type-checked candidates:
	(-0.066) rintro rfl
	(-0.230) rintro ⟨d, rfl⟩
	(-0.301) rintro ⟨⟨a, rfl⟩, b, rfl⟩
	(-0.307) rintro ⟨rfl, rfl⟩
--- current:
	theorem thm1 (a b c : Nat) : a + b = c → a ≤ c := by 
	rintro rfl


100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.36s/it]

--- type-checked candidates:
	(-0.109) apply Nat.le_add_right
	(-0.173) exact Nat.le_add_right _ _


{'theorem_statement': 'theorem thm1 (a b c : Nat) : a + b = c → a ≤ c := by {}',
 'proof': ['rintro rfl', 'apply Nat.le_add_right'],
 'state': {'env': 0},
 'score': 0.17478330433368683,
 'success': True}